<a href="https://colab.research.google.com/github/Seowoo-Jung/ESAA/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C(8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (6주차) 8. 파이썬 추천 시스템 패키지 - Surprise (p.607 ~ 624)

### 1) Surprise 패키지 소개

- 장점: 다양한 추천 알고리즘(아이템 기반, 잠재요인 협업필터링)을 쉽게 사용할 수 있음. fit, predict()같은 함수들 이름이 사이킷 런 모델링과 비슷


### 2) Surprise 를 이용한 추천 시스템 구축

[Example]
- train, test 분리 후 잠재요인협업필터링(SVD 행렬분해) 수행.

In [19]:
! pip install scikit-surprise

In [20]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [21]:
## 데이터 로딩 : Surprise에 Dataset 클래스를 통해서만 가능
# MovieLens 사이트에서 제공하는 과거 데이터 세트를 가져온다.
data = Dataset.load_builtin('ml-100k') # 이거 실행하면 아카이브 사이트로부터 내려받아 로컬 디렉터리에 저장하게됨
#수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [22]:
# 데이터 파일 분리문자: \t
# 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야함. (데이터 다운받은 그대로 변환)
# (아이템 아이디를 칼럼명으로 변환하여 사용자-아이템 평점 행렬을 변환함)
# Surprise는 자체적으로 로우 레벨의 데이터를 칼럼레벨 데이터로 변경하기 때문에 로우 레벨 데이터 세트로 적용

algo=SVD()
algo.fit(trainset) # 학습

In [23]:
# SVD 행렬로 분해 후 추천 예측(test, predict)
## test 함수를 이용한 예측
predictions = algo.test(testset) # 입력된 데이터 세트에 대한 추천 데이터 세트 반환.
print('prediction type: ', type(predictions), ' size:', len(predictions)) # size는 데이터 세트 크기와 같음. 
print('prediction 결과의 최초 5개 추출') 
predictions[:5]

prediction type:  <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.7078770868862008, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.9364634601237563, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.288764788884727, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.371276048305824, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.7264836624995277, details={'was_impossible': False})]

 - uid: 개별 사용자 id
 - iid: 영화 아이디 
 - r_ui: 실제 평점
 - est: 예측 평점 
 - details: 추천을 예측할 수 없는 경우 로그용 ('was_impossible': True)이면 예측값 생성 할 수 없음.

In [24]:
# prediction 객체의 속성에 접근
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.7078770868862008),
 ('882', '291', 3.9364634601237563),
 ('535', '507', 4.288764788884727)]

In [25]:
## predict를 이용한 추천 예측: 개별 사용자의 아이템에 대한 추천 평점
# 사용자 아이디, 아이템 아이디는 "문자열"로 입력해야함
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.01   {'was_impossible': False}


test() 는 predict() 여러번 반복한 것.

In [26]:
# 실제값과 예측 평점 사이의 차이(성능)
accuracy.rmse(predictions)

RMSE: 0.9481


0.9481426733566418

### 3) Surprise 주요 모듈 소개

- Dataset

< OS 파일 데이터를 Suprise로 로딩: Dataset.load_from_file>

: low-level dataset 만 적용할 수 있음.

: user_id, item_id, rating 순으로 되어있어야 하고 네번째 칼럼부터는 로딩하지 않음.

In [27]:
import pandas as pd

ratings = pd.read_csv("/ratings.csv")
# ratings_noh.csv 파일로 언로드 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv("/ratings_noh.csv", index=False, header=False)

In [28]:
# ratings_noh.csv 임포트 
# Reader 클래스를 이용하여 파싱 포맷지정.(데이터는 콤마로 분리되어있음)
# 칼럼명, 분리문자, 최소~최대평점을 입력하여 데이터 파일을 파싱하면서 로딩
from surprise import Reader

reader=Reader(line_format='user item rating timestamp', sep=',',rating_scale=(0.5,5))
data=Dataset.load_from_file('/ratings_noh.csv', reader=reader)

- Surprise 데이터 세트를 쓰는 경우 무비렌즈 데이터 형식을 따르므로 외부에서 데이터를 가져올때 Reader클래스를 설정하여 형식을 지정해줘야한다.

<Reader 클래스의 주요 생성 파라미터>

- line_format: 공백으로 분리하여 칼럼 인식
- sep: 칼럼을 분리하는 분리자. default='\t'
- raing_scale: 평점의 최대, 최소, default=(1,5)

In [29]:
## 예측
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

#수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo=SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

< 판다스에서 Surprise 데이터 세트로 로딩 : Dataset.load_from_df() 사용>

- 순서를 반드시 지킨다! (사용자 id, 아이템 id, 평점)





In [30]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv("/ratings.csv")
reader=Reader(rating_scale=(0.5, 5.0))

# ratings Dataframe 에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야함.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# ratings[['userId', 'movieId', 'rating']]: 이렇게 설정하면 어차피 안에 내용만 들어가게되니까 header제거 필요없네..
trainset, testsets = train_test_split(data, test_size=0.25, random_state=0)

algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.0388


1.0387858758781252

### 4) Surprise 추천 알고리즘 클래스

: SVD, KNNBasic, BaselineOnly

- SVD: SVD의 비용함수는 베이스라인의 편향성을 감안한 평점 예측에 정규화를 적용한 것!

--> 베이스 라인?

베이스라인 모형(baseline model)은 사용자 아이디 u, 상품 아이디 i, 두 개의 카테고리 값 입력에서 평점 r(u,i)의 예측치 r^(u,i) 을 예측하는 가장 단순한 회귀분석모형으로 다음과 같이 사용자와 상품 특성에 의한 평균 평점의 합으로 나타난다.

r^(u,i)=μ+b(u)+b(i) -->사용자 취향(편향) 적용 + 과최적화를 피하기 위해 정규화 적용.

블로그 참고: https://datascienceschool.net/03%20machine%20learning/07.01%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C.html#id4

### 5) 베이스라인 평점

: 한 개인의 성향을 반영해 아이템 평가에 편향성(bias)요소를 반영하여 평점을 부과하는 것

: 전체 평균 평점 

+사용자 편향 점수 (사용자별 아이템 평점 평균 - 전체 평균 평점)

+아이템 편향 점수 (아이템별 평점 평균 - 전체 평균 평점)

### 6) 교차 검증과 하이퍼 파라미터 튜닝

: cross_validate()를 이용!

In [31]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/ratings.csv')
reader=Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8716  0.8723  0.8755  0.8753  0.8765  0.8742  0.0019  
MAE (testset)     0.6693  0.6684  0.6747  0.6742  0.6732  0.6720  0.0026  
Fit time          4.91    4.85    4.88    4.85    5.12    4.92    0.10    
Test time         0.15    0.29    0.16    0.32    0.15    0.21    0.07    


{'fit_time': (4.906571388244629,
  4.847833871841431,
  4.8841328620910645,
  4.8467795848846436,
  5.116687536239624),
 'test_mae': array([0.6693166 , 0.66840321, 0.67465061, 0.67418395, 0.67322438]),
 'test_rmse': array([0.87161263, 0.87233666, 0.87549318, 0.87527298, 0.87646657]),
 'test_time': (0.15309619903564453,
  0.29187750816345215,
  0.15668916702270508,
  0.3162422180175781,
  0.152726411819458)}

In [32]:
# 파라미터 최적화
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

# CV를 3개 폴드 세트로 지정, 성능 평가를 rmse, mad로 수행되도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8769404977431111
{'n_epochs': 20, 'n_factors': 50}


### 7) Surprise를 이용한 개인화 영화 추천 시스템 구축

: 잠재 요인 협업 필터링 기반의 개인화된 영화 추천. (사용자가 평점을 매개지 않은 영화 중 개인 취향에 적절한 영화 추천!

- 데이터를 분리하지 않고 전체를 학습 데이터로 사용한다. But, Surprise는 데이터 세트를 TrainSet 클래스 객체로 변환하지 않으면 학습할 수 없다.

In [33]:
## 오류
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발행합니다.
data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo=SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

따라서 전체 데이터를 학습하려면 DatasetAutoFolds 클래스 사용!

In [34]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5) )
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds=DatasetAutoFolds(ratings_file='/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함.
trainset=data_folds.build_full_trainset() 

algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [35]:
# userid=9가 평점을 매기지 않은 영화를 movieId 42로 선정하고 예측 평점을 계산
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies=pd.read_csv('/movies.csv')

# userId=09의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인.
movieIds=ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 형과 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 형과 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [38]:
# 9번 사람의 42번 영화에 대한 예상 평점
uid=str(9)
iid=str(42)

pred=algo.predict(uid, iid, verbose=True)
# verbose: T: 로깅을 출력할지 말지에 대한 옵션
# 이거 true해야 결과가 나옴

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


- 사용자가 평점을 매기지 않은 전체 영화를 추출하고 예측 평점 순으로 영화를 추천

1. 추천대상이 되는 영화 추출: get_unseen_movies() 정의

In [39]:
def get_unseen_surprise(ratings, movies, userId):
  #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성.
  total_movies=movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [44]:
## SVD를 이용하여 높은 예측 평점을 가진 순으로 영화 추천!
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise 의 Preditions 객체를 원소로 가지고 있음.
  # [Predition(uid='9', iid='1', est=3.69), Preditions(uid='9', iid='2', est=2.98),,, ]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey 함수를 정의함.
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
  def sortkey_est(pred):
    return pred.est # est 값만 뽑아서 return

  # sortkey_est() 반환값의 내림 차순으로 정렬수행하고 top_n개의 최상위 값 추출,
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions=predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating=[pred.est for pred in top_predictions]
  top_movie_titles=movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [ (id, title, rating) for id, title, rating in \
                     zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

In [45]:
unseen_movies=get_unseen_surprise(ratings, movies, 9)
top_movie_preds=recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print("##### Top-10 추천 영화 리스트 #####")
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
